In [1]:
import requests
disease_news = requests.get('https://raw.githubusercontent.com/ywchiu/cdc_course/master/data/news.txt')
disease_news.encoding = "big5"
disease_news = disease_news.text

In [2]:
import re
re.split('、|；|，|）|（|。', disease_news)

['新北市出現當地今年首起登革熱群聚\u3000憂疫情擴大\n\n疾管署提醒',
 '赴東南亞要注意防蚊\n    \n新北市出現當地今年首例登革熱群聚事件',
 '衛福部疾管署表示',
 '患者為家住鶯歌區的一對夫妻',
 '近期沒有出國',
 '但卻先後感染登革熱發病',
 '疾管署表示',
 '檢驗夫妻感染的登革熱病毒株為接近越南登革熱流行型別',
 '其居住社區可能潛藏境外移入病例',
 '但兩人就醫2至4次才被確診',
 '恐導致社區潛在個案無法快速被掌握及通報',
 '疫情可能擴大',
 '\n疾管署副署長羅一鈞表示',
 '這對夫妻約50多歲',
 '太太先於9月20日發病',
 '出現發燒',
 '肌肉痛',
 '關節痛',
 '倦怠等症狀',
 '之後後丈夫也發病',
 '但直到昨日才確診為第一型登革熱',
 '而妻子自昨日確診時已是第4次就醫',
 '從發病到被通報疑似為登革熱',
 '病例隱藏期已歷經7天',
 '\n羅一鈞表示',
 '登革熱埃及斑蚊較常在南部的高雄',
 '屏東及台南出現大流行',
 '北部',
 '中部的病媒蚊則以白線斑蚊為主',
 '未曾有過登革熱大流行',
 '因此相較使北部醫療院所較少有診治登革熱經驗',
 '病例隱藏期延長',
 '恐導致社區潛在個案未能快速被掌握及通報',
 '提升疫情傳播風險',
 '\n羅一鈞表示',
 '目前已針對患者居家',
 '活動地周邊進行病媒蚊及孳生源清除',
 '2名患者目前正住院治療中',
 '同住家人暫無疑似症狀',
 '\n羅一鈞說',
 '由於南部較常出現登革熱流行疫情',
 '掌握感染從者從發病到被通報',
 '病例隱藏期可掌握在1.5天',
 '為做好疫情控制',
 '至少應在3天內通報',
 '\n根據疾管署統計',
 '今年初截至昨日為止',
 '國內已有5例登革熱本土病例',
 '234例境外移入病例',
 '含2例死亡',
 '',
 '最近東南亞寮國',
 '越南及泰國',
 '登革熱疫情呈上升或處於高水平',
 '越南已累積了12萬5千例患者',
 '比去年同期增加逾40%',
 '寮國逾7700例',
 '提醒民眾到東南亞旅行應做好防蚊措施',
 '(黃仲丘／台北報導)']

In [3]:
import re
from collections import Counter

def ngram(sentence, n = 2):
    words = []
    for i in range(0, len(sentence) - n + 1):
        words.append(sentence[i:i+n])
    return words

def removeKey(text, keyword):
    for key in keyword:
        text = text.replace(key,'')
    return text

def longTermFirst(article, keywords, threshold):
    delimiter = '，|、|「|」|。|\n'
    sentence_ary = re.split(delimiter, article)

    for k in range(4,1,-1):
        # 產生n-gram 詞
        terms    = []
        for sentence in sentence_ary:
            sentence = removeKey(sentence, keywords)
            terms.extend(ngram(sentence, k))

        # 統計n-gram, 將超過一定次數(閥值)的資料取出
        c = Counter(terms)
        for word, cnt in c.items():
            if cnt >= threshold:
                keywords.append(word)
                
    # 回傳關鍵字詞
    return keywords

In [4]:
article = disease_news
keywords = []
threshold = 4

In [5]:
important_words = longTermFirst(article, keywords, threshold)
important_words

['登革熱', '疾管署', '羅一鈞', '出現', '疫情', '表示', '患者', '發病', '流行', '病例', '掌握', '通報']

In [6]:
with open('localdict.txt', 'a', encoding = 'UTF-8') as f:
    for word in important_words:
        f.write('{} {} {}\n'.format(word, 100, 'n'))

In [7]:
import jieba
jieba.load_userdict('localdict.txt')
words = [w for w in jieba.cut(article)]
from collections import Counter
words = [w for w in jieba.cut(article)]
c = Counter(words)
c.most_common(10)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 1.143 seconds.
Prefix dict has been built succesfully.


[('，', 40),
 ('登革熱', 13),
 ('\n', 9),
 ('、', 9),
 ('疫情', 6),
 ('為', 6),
 ('病例', 6),
 ('。', 6),
 ('出現', 5),
 ('疾管署', 5)]

In [8]:
import re
import pandas
common_terms = []
for k, v in c.most_common(100):
    if len(k) >= 2 and re.match('[\u4E00-\u9FA5]+', k):
        common_terms.append({'keyword': k, 'count': v})

In [9]:
df = pandas.DataFrame(common_terms).head(3)
df

,count,keyword
0,13,登革熱
1,6,疫情
2,6,病例
